## 1. Install necessary packages

In [ ]:
! pip install  azureml-sdk[automl,explain,notebooks]==1.0.43 matplotlib 

## 2. Load workspace from conf.json

In [ ]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None


from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig


In [ ]:
ws = Workspace.from_config()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

## 3. Print the exiting compute in the workspace

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

compute_target_name='cpucluster'
compute = ws.compute_targets
compute_target=compute[compute_target_name]
print(compute_target.serialize())

## 4. Upload data to blob storage for later on experiments

In [ ]:
ds = ws.get_default_datastore()
ds.upload_files(['./data/nyc_energy.csv'], target_path='exp_data', overwrite=True, show_progress=True)

from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='exp_data', 
                   path_on_compute='/tmp/azureml_runs',
                   mode='download', # download files from datastore to compute target
                   overwrite=False)